In [44]:
import numpy as np
import operator
import pickle
from scipy.spatial import KDTree
import tifffile

In [38]:
def find_indices(array, condition=operator.eq, value=1):
    # Return list of indices for cells meeting a logical condition
    indices = np.argwhere(condition(array, value))
    return [tuple(idx) for idx in indices]

In [3]:
# Load preprocessed 3d data
data_3d = np.load('output/pvd_test.npy')

# Print descriptives
print(f"min: {np.amin(data_3d)} max:{np.amax(data_3d)} shape:{data_3d.shape} type:{type(data_3d)} ")

min: 0 max:1 shape:(188, 2044, 2042) type:<class 'numpy.ndarray'> 


In [40]:
data_idx = find_indices(data_3d)
print(f"number of voxels: {len(data_idx)}")

number of voxels: 1256635


In [64]:
# Load list of outer+core segments
with open('output/outer_segments.pkl', 'rb') as f:
    segments = pickle.load(f)
print(f"number of outer segments: {len(segments[2])}")

number of outer segments: 52


In [74]:
def assign_segments_to_indices(indices, segments):
    # Flatten the segments list and create a mapping of coordinates to segment indices
    flattened_segments = []
    coord_to_segment = {}
    
    for segment_index, segment in enumerate(segments):
        for coord in segment:
            flattened_segments.append(coord)
            coord_to_segment[tuple(coord)] = segment_index
    
    # Create a KDTree from the flattened segment coordinates
    tree = KDTree(flattened_segments)
    
    # List to store the segment assignments
    segment_assignments = []

    # Enumerate through the list of indices
    for idx in indices:
        # Query the KDTree for the nearest segment to the current index
        _, nearest_flattened_index = tree.query(idx)
        
        # Retrieve the segment index from the mapping
        nearest_segment_index = coord_to_segment[tuple(flattened_segments[nearest_flattened_index])]
        
        # Append the segment index to the assignments list
        segment_assignments.append(nearest_segment_index)

    return segment_assignments

In [75]:
segment_assignments = assign_segments_to_indices(data_idx, segments)

In [89]:
def label_data_for_imagej_color(indices, segment_assignments, shape, num_segments):
    # Create an empty array to hold the labeled data (3 channels for RGB)
    labeled_array = np.zeros((*shape, 3), dtype=np.uint8)
    
    # Generate unique colors for each segment
    np.random.seed(0)  # For reproducibility
    colors = np.random.randint(0, 255, size=(num_segments, 3), dtype=np.uint8)
    
    # Assign colors to the corresponding coordinates
    for coord, segment in zip(indices, segment_assignments):
        labeled_array[coord[0], coord[1], coord[2]] = colors[segment]
    
    return labeled_array


shape = data_3d.shape
num_segments = len(set(segment_assignments))+2
labeled_array_color = label_data_for_imagej_color(data_idx, segment_assignments, shape, num_segments)

# Save the labeled array as a TIFF file
tifffile.imwrite('labeled_data_color.tif', labeled_array_color)